In [1]:
import numpy as np
import pandas as pd

In [2]:
features=pd.read_csv("item_features.csv")

In [3]:
features["value"]=1

In [4]:
len(features)

471751

In [5]:
features.head()

,item_id,feature_category_id,feature_value_id,value
0,2,56,365,1
1,2,62,801,1
2,2,68,351,1
3,2,33,802,1
4,2,72,75,1


In [6]:
feat_categories=features["feature_category_id"].unique()
print("min:",min(feat_categories),"  max:",max(feat_categories))

min: 1   max: 73


In [7]:
feat_values=features["feature_value_id"].unique()
print("min:",min(feat_values),"  max:",max(feat_values))

min: 1   max: 905


In [8]:
feat_category_count=features.groupby("feature_category_id")["value"].count()

In [9]:
print("min:",min(feat_category_count),"  max:",max(feat_category_count))

min: 1   max: 23691


In [10]:
feat_category_count[feat_category_count<500]

feature_category_id
9     137
13    137
27      1
54    341
Name: value, dtype: int64

In [11]:
feat_category_count[feat_category_count>20000]

feature_category_id
7     20416
47    23691
50    23077
56    23691
61    22512
68    23038
69    20450
72    20499
Name: value, dtype: int64

In [12]:
feats=features.groupby(["feature_category_id","feature_value_id"])["value"].count().reset_index()

In [13]:
feats=feats.sort_values(by="value",ascending=False)
feats.head()

,feature_category_id,feature_value_id,value
637,56,365,19472
878,72,75,12582
716,61,706,12353
190,17,378,11939
743,63,861,11651


In [14]:
feats.tail()

,feature_category_id,feature_value_id,value
338,30,641,1
351,30,827,1
362,31,306,1
375,33,530,1
0,1,60,1


In [15]:
feats=feats.reset_index()
feats["feature_idx"]=feats.index+1

In [16]:
feats

,index,feature_category_id,feature_value_id,value,feature_idx
0,637,56,365,19472,1
1,878,72,75,12582,2
2,716,61,706,12353,3
3,190,17,378,11939,4
4,743,63,861,11651,5
...,...,...,...,...,...
899,338,30,641,1,900
900,351,30,827,1,901
901,362,31,306,1,902
902,375,33,530,1,903


In [17]:
feats[["feature_category_id","feature_value_id","feature_idx"]].to_csv("feature_map.csv",index=False)

In [18]:
feats=feats[["feature_category_id","feature_value_id","feature_idx","value"]]
feats.rename(columns={"value":"count"},inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [19]:
map=pd.read_csv("map.csv")

In [20]:
features_mapped=features.merge(map,how="left",on="item_id")

In [21]:
features_mapped=features_mapped[["code","feature_category_id","feature_value_id"]]
features_mapped.rename(columns={"code":"item_id"},inplace=True)

In [22]:
features_mapped.to_csv("features_mapped.csv",index=False)

In [23]:
features_mapped=features_mapped.merge(feats,how="left",on=["feature_category_id","feature_value_id"])

In [24]:
features_mapped=features_mapped[["item_id","feature_idx"]]

In [25]:
features_mapped

,item_id,feature_idx
0,19021,1
1,19021,23
2,19021,16
3,19021,447
4,19021,2
...,...,...
471746,19020,16
471747,19020,363
471748,19020,75
471749,19020,46


In [26]:
features_mapped.to_csv("simplified_features.csv",index=False)

In [29]:
session_train=pd.read_csv("train_sessions.csv")

In [32]:
session_train=session_train[["session_id","item_id"]]

In [33]:
len(session_train)

4743820

In [34]:
URM_df=session_train.merge(features_mapped,how="right",on="item_id")

In [35]:
len(URM_df)

102883382

In [36]:
URM_df["count"]=1

In [38]:
URM_df_count_feats=URM_df.groupby(by=["session_id","feature_idx"])["count"].sum()

In [41]:
URM_df_count_feats=URM_df_count_feats.reset_index()
URM_df_count_feats.head()

,session_id,feature_idx,count
0,3.0,1,2
1,3.0,7,2
2,3.0,8,2
3,3.0,24,2
4,3.0,28,2


In [44]:
URM_df_count_feats=URM_df_count_feats.astype(int)

In [48]:
URM_df_count_feats.to_csv("session_URM.csv",index=False)